In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import os
import random
from datetime import datetime

import cv2
import keras
import numpy as np
import pandas as pd
from keras import backend as K
from keras.applications import *
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.datasets import fashion_mnist
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import *
from keras.utils.vis_utils import model_to_dot
from tqdm import tqdm
from keras import backend as K
from skimage.transform import resize

from scipy import misc

Using TensorFlow backend.


In [2]:
def load_data():
    num_classes = 10
    height,width = 128, 128
        
    (train, train_l), (test, test_l) = fashion_mnist.load_data()

    # convert class vectors to binary class matrices
    y = keras.utils.to_categorical(train_l, num_classes)
    y_test = keras.utils.to_categorical(test_l, num_classes)
    
    
    # use this in future
    # X_train = np.array([resize(x, (height,width)).astype(float) for x in tqdm(iter(X_train.astype(int)))])/255.
    
    train = train.reshape((-1,28,28))
    train = np.array([misc.imresize(x, (height,width)) for x in tqdm(iter(train))])
    test = test.reshape((-1,28,28))
    test = np.array([misc.imresize(x, (height,width)) for x in tqdm(iter(test))])

    x = np.stack((train, train, train), axis=3)
    x_test = np.stack((test, test, test), axis=3)
    
    print(x.shape)
    # devide into train and validation 
    dvi = int(train.shape[0] * 0.9)
    x_train = x[:dvi, :, :, :]
    y_train = y[:dvi, :]
    x_val = x[dvi:, :, :, :]
    y_val = y[dvi:, :]
    
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_val.shape[0], 'validation samples')
    print(x_test.shape[0], 'test samples')

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)


In [3]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data()

0it [00:00, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
60000it [00:06, 9804.27it/s]
0it [00:00, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
10000it [00:00, 10175.19it/s]


(60000, 128, 128, 3)
x_train shape: (54000, 128, 128, 3)
54000 train samples
6000 validation samples
10000 test samples


In [5]:
def run(model_name, lr, optimizer, epoch, patience, batch_size):
    # Loading Datasets
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = load_data()

    # Compute the bottleneck feature
    
    n_class = y_test.shape[1]
    input_shape = (height, width, 3)
    weights=None

    def get_features(MODEL, data=x_train):
        cnn_model = MODEL(
            include_top=False,
            input_shape=input_shape,
            weights=weights)

        inputs = Input(input_shape)
        x = inputs
        x = Lambda(preprocess_input, name='preprocessing')(x)
        x = cnn_model(x)
        x = GlobalAveragePooling2D()(x)
        cnn_model = Model(inputs, x)

        features = cnn_model.predict(data, batch_size=32, verbose=1)
        return features

    def fine_tune(MODEL,
                  model_name,
                  optimizer,
                  lr,
                  epoch,
                  patience,
                  batch_size,
                  X=x_train):
        # Fine-tune the model
        print("\n\n Fine tune " + model_name + " : \n")
        
        if weights != None:
            try:
                model.load_weights(model_name + '.h5')
                print('Load ' + model_name + '.h5 successfully.')
            except:
                try:
                    model.load_weights('fc_' + model_name + '.h5', by_name=True)
                    print('Fail to load ' + model_name + '.h5, load fc_' +
                          model_name + '.h5 instead.')
                except:
                    print(
                        'Start computing ' + model_name + ' bottleneck feature: ')
                    features = get_features(MODEL, X)

                    # Training models
                    inputs = Input(features.shape[1:])
                    x = inputs
                    x = Dropout(0.5)(x)
                    x = Dense(n_class, activation='softmax', name='predictions')(x)
                    model_fc = Model(inputs, x)
                    model_fc.compile(
                        optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])
                    h = model_fc.fit(
                        features,
                        y_train,
                        batch_size=128,
                        epochs=5,
                        validation_split=0.1)

                    model_fc.save('fc_' + model_name + '.h5', 'w')
                    
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            horizontal_flip=True)
        val_datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input)


        inputs = Input(input_shape)
        x = inputs
        cnn_model = MODEL(
            include_top=False, input_shape=input_shape, weights=None)
        x = cnn_model(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        x = Dense(n_class, activation='softmax', name='predictions')(x)
        model = Model(inputs=inputs, outputs=x)


        # for layer in model.layers[:114]:
        #     layer.trainable = False

        
        
        
        print("\n " + "Optimizer=" + optimizer + " lr=" + str(lr) + " \n")
        
        if optimizer == "Adam":
            model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
        elif optimizer == "SGD":
            model.compile(
                loss='categorical_crossentropy',
                optimizer=SGD(lr=lr, momentum=0.9, nesterov=True),
                metrics=['accuracy'])

        class LossHistory(keras.callbacks.Callback):
            def on_train_begin(self, logs={}):
                # self.val_losses = []
                self.losses = []

            def on_epoch_end(self, batch, logs={}):
                # self.val_losses.append(logs.get("val_loss"))
                self.losses.append((logs.get('loss'), logs.get("val_loss")))
        
#         model.summary()
        history = LossHistory()

        early_stopping = EarlyStopping(
            monitor='val_loss', patience=patience, verbose=1, mode='auto')
        checkpointer = ModelCheckpoint(
            filepath=model_name + '.h5', verbose=0, save_best_only=True)
        h2 = model.fit_generator(
            datagen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=len(x_train) / batch_size,
            validation_data=val_datagen.flow(
                x_val, y_val, batch_size=batch_size),
            validation_steps=len(x_val) / batch_size,
            epochs=epoch,
            callbacks=[early_stopping, checkpointer, history])
        
        with open(model_name + ".csv", 'a') as f_handle:
            np.savetxt(f_handle, history.losses)
        
    list_model = {
        "Xception": Xception,
        "InceptionV3": InceptionV3,
        "InceptionResNetV2": InceptionResNetV2,
        "VGG16": VGG16,
        "MobileNet": MobileNet
    }
    print(list_model[model_name])
    fine_tune(list_model[model_name], model_name, optimizer, lr, epoch,
              patience, batch_size, x_train)

In [ ]:
run("MobileNet", 1e-2, "Adam", 10000, 5, 64)

0it [00:00, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
60000it [00:05, 10036.79it/s]
0it [00:00, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
10000it [00:00, 10153.36it/s]


(60000, 128, 128, 3)
x_train shape: (54000, 128, 128, 3)
54000 train samples
6000 validation samples
10000 test samples
<function Xception at 0x7fe99187c5f0>


 Fine tune Xception : 


 Optimizer=Adam lr=3e-05 

Epoch 1/10000
438/843 [==============>...............] - ETA: 4:00 - loss: 0.4790 - acc: 0.8264

In [ ]:
# Training preprocess visualization
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')